In [6]:
import requests
import pandas as pd

# Fetch from Eurostat API
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tec00120?format=JSON"
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Extract metadata
values = data["value"]
geo_labels = data["dimension"]["geo"]["category"]["label"]
geo_indices = data["dimension"]["geo"]["category"]["index"]
time_labels = data["dimension"]["time"]["category"]["label"]
time_indices = data["dimension"]["time"]["category"]["index"]
geo_size = data["size"][3]
time_size = data["size"][4]

# Define EU countries (and EU27)
eu_codes = [
    'EU27_2020', 'BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR',
    'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT',
    'RO', 'SI', 'SK', 'FI', 'SE'
]

# Reconstruct mapping and extract records
records = []
for idx_str, val in values.items():
    idx = int(idx_str)
    geo_pos = idx % geo_size
    time_pos = (idx // geo_size) % time_size

    geo_code = next(k for k, v in geo_indices.items() if v == geo_pos)
    year = next(k for k, v in time_indices.items() if v == time_pos)

    if geo_code in eu_codes and int(year) >= 2015:
        records.append({
            "Country": geo_labels[geo_code],
            "Year": year,
            "Price Level Index": val
        })

# Create and pivot DataFrame
df = pd.DataFrame(records)
df_pivot = df.pivot(index="Country", columns="Year", values="Price Level Index").reset_index()

# Save to CSV
filename = "eu_comparative_price_levels_2015_2023.csv"
df_pivot.to_csv(filename, index=False)

# Print download message
print(f"downloaded")







downloaded
